In [1]:
#------------------------------importing library-----------------------------------------
from dateutil.parser import parse
import numpy as np
import pandas as pd
from dateutil.parser import parse
import pickle
from bpemb import BPEmb
import pytesseract
import cv2
import glob
import matplotlib.pyplot as plt
from PIL import Image
from bs4 import BeautifulSoup
import re
import networkx as nx

import torch
import torch.nn.functional as F
from torch_geometric.nn import ChebConv
from torch.utils.data import Dataset 
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from scipy.sparse import csr_matrix
from torch_geometric.utils import convert

import torch.nn as nn
from timeit import default_timer as timer

In [2]:
def SaveData(data,path):
    with open(path,'wb') as file:
        pickle.dump(data,file)

def LoadData(path):
    with open(path, 'rb') as file:
        return pickle.load(file)
    
# boolean_feature =['isDate','isZipCode','isKnownCity','isKnownDept','isKnownCountry', # below is nature feature
# 'isAlphabetic','isNumeric','isAlphaNumeric','isNumberwithDecimal','isRealNumber','isCurrency','hasRealandCurrency','mix/mixc']

def is_date_o(string, fuzzy=False):
    print(string)
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False
    
def is_date(string, fuzzy=False):
#     print(string)
    try:
        if is_number(string):
            if(int(string) > 9999):
                return False
        elif is_numberwithDecimal(string):
            return False     
    except:
        if is_numberwithDecimal(string):
            return False
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False
    
def is_zipcode(code_str,database):
    if is_number(code_str):
        if code_str in database:
            return True
        else:
            return False
    else :
        return False
    
def is_knowCity(city_str,database):
    if is_alphbetic(city_str):
        if city_str in database:
            return True
        else:
            return False
    else :
        return False

def is_knownDept(dept_str,database):
    if is_alphbetic(dept_str):
        if dept_str in database:
            return True
        else:
            return False
    else :
        return False

def is_knownCountry(country_str,database):
    if is_alphbetic(country_str):
        if country_str in database:
            return True
        else:
            return False
    else :
        return False    
    
# creatring fucntion for nature vector
def is_alphbetic(alph_str):
    return bool(re.match('^[a-zA-Z]+$', alph_str))

def is_number(num_str):
    return bool(re.match('^[0-9]+$', num_str))

def is_alphaNumeric(alphNum_str):
    return bool(re.match('^[a-zA-Z0-9]+$', alphNum_str))

def is_numberwithDecimal(num_str):
    
    if is_realNumer(num_str):  # checking if it is in numeric form
        num = num_str.split('.') 
        if len(num)>=2 and len(num[-1])>=1: # length after decimal point must be >=1
            return True
        else:
            return False
    else:   
        return False

def is_realNumer(num_str):
    try: 
        float(num_str)
    except ValueError: 
        return False
    return True
    
def is_currency(curr_str):
    curr_symbol='¥$€£₹'
    curr_str =list(curr_str)
    if curr_str[0] in curr_symbol or curr_str[-1] in curr_symbol:
        return True
    else:
        return False
    
def is_hasRealandCurrency(curr_str):
    currency = list(curr_str)
    curr_nature ='-+'
    if currency[0] in curr_nature:
        if is_currency(''.join(currency[1:])):
            return True
        else:
            return False
    else:
        return False
        
def create_nature_vector(word):
    nature=[] # 8 -dimentional vector 
    nature.append(is_alphbetic(word))
    nature.append(is_number(word))
    nature.append(is_alphaNumeric(word))
    nature.append(is_numberwithDecimal(word))
    nature.append(is_realNumer(word))
    nature.append(is_currency(word))
    nature.append(is_hasRealandCurrency(word))    
    if not any(nature):
        mix =True
    else:
        mix =False
    nature.append(mix)
    return nature 

def create_Boolean_feature (word,database):
    # boolean_feature =['isDate','isZipCode','isKnownCity','isKnownDept','isKnownCountry', # below is nature feature
    # 'isAlphabetic','isNumeric','isAlphaNumeric','isNumberwithDecimal','isRealNumber',
    #'isCurrency','hasRealandCurrency','mix/mixc']

    date = is_date(word)
    zipcode = is_zipcode(word,database[0])
    city = is_knowCity(word,database[1])
    dept = is_knownDept(word,database[2])
    country = is_knownCountry(word,database[3])
    nature_vect = create_nature_vector(word) 
#    print(['isDate','isZipCode','isKnownCity','isKnownDept','isKnownCountry', # below is nature feature
#           'isAlphabetic','isNumeric','isAlphaNumeric','isNumberwithDecimal','isRealNumber',
#    'isCurrency','hasRealandCurrency','mix/mixc'])
    return np.array([date,zipcode,city,dept,country]+nature_vect)

#https://github.com/bheinzerling/bpemb/blob/master/bpemb/bpemb.py
def create_text_feature(word, bpemb_encoder):
    ids = bpemb_encoder.encode_ids(word)
#    print(ids)
    if len(ids)==1:
        embding = bpemb_encoder.vectors[ids]
        embding =np.concatenate((embding.flatten(), np.zeros((200))))
    elif len(ids)==2:
        embding = bpemb_encoder.vectors[ids]
        embding =np.concatenate((embding.flatten(), np.zeros((100))))
    elif len(ids)==3:
        embding = bpemb_encoder.vectors[ids]
        embding=embding.flatten()
    else:
#        embding = bpemb_encoder.vectors[ids]  # need to take care of case having subword length >3
#        embding = np.split(embding, 4)[0:3]
         embding = np.zeros((300))   
#    print(len(embding))
    return embding

def details(data):
    print('data type :',type(data))
    try:
        print('data shape : ', data.shape)
    except:
        print('data lenght : ',len(data))

def create_feature_vector(word_dict,num_feature):
     database = LoadData(database_path) #todo 
     bpemb_encoder = BPEmb(lang="en", dim=100)
     feature_vector=[]
     for idx, word_info in word_dict.values():
#         print(word_info)
        word = word_info[1].strip()
        bool_feature = create_Boolean_feature(word,database)
        text_feature = create_text_feature(word,bpemb_encoder)
        feature =np.concatenate((bool_feature , num_feature[idx] , text_feature))
        feature_vector.append(feature)
     
     return feature_vector
    
def extract_bbox_info(img_path):
    hocr_data = pytesseract.image_to_pdf_or_hocr(img_path, extension='hocr')
    soup = BeautifulSoup(hocr_data)
    spans = soup.find_all('span', attrs={'class':'ocrx_word'})  # getting the attributes 
    word_bbox_list =[]  
    bbox_width=[]
    for span in spans:
        bbox_str=span['title']  # geting bounding box inforamtion 
        bbox_str =bbox_str.split(';')
        bbox =bbox_str[0].split()[1:]
    #     x,y,w,h= *bbox
    #     word_info_list.append([span['id'],span.string,[bbox[1].split()[-1],*bbox]])
        word_bbox_list.append([span['id'],span.string,
                               [int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])]
                              ])
        bbox_width.append((int(bbox[2])-int(bbox[0])))
#         word_info_list.sort(key = lambda sort_value: (sort_value[2][1],sort_value[2][0]))
#     word_info_list.sort(key = lambda sort_value: sort_value[2][1])
    
    return word_bbox_list, bbox_width

def remove_noise(bbox_width, word_list):
    mean=np.mean(bbox_width) # mean of the width of bounding box
    sd=np.std(bbox_width)  #stanered deviation of the width of the bounding box
    sd=round(sd) #Roinding to integer value and find the threshold value
#     print(mean,sd)
    def noise_removal_finction(box_info):
        box_width = box_info[2][2] - box_info[2][0]
#         print(box_width)
        if  (mean*2-sd) <=box_width < mean+sd:
            if box_info[1] !=' ' and box_info[1] !='-':
                return box_width
    
    filtered_word_list = filter(noise_removal_finction, word_list)
    return list(filtered_word_list)

def remove_noise_info(bbox_width, word_list):
    mean=np.mean(bbox_width) # mean of the width of bounding box
    sd=np.std(bbox_width)  #stanered deviation of the width of the bounding box
    sd=round(sd) #Roinding to integer value and find the threshold value
#     print(mean,sd)
    filtered_word_list=[]
    remove_word_list=[]
    for box_info in word_list:
        box_width = box_info[2][2] - box_info[2][0]
#         print(box_width)
        if  (mean*2-sd) <=box_width < mean+sd:
            if box_info[1] !=' ':
                filtered_word_list.append(box_info)
            else:
                remove_word_list.append(box_info)
        else:
            remove_word_list.append(box_info)
    
    return remove_word_list, filtered_word_list

def Line_Formation(word_list):
    word_list.sort(key =lambda sort_on_top : sort_on_top[2][1]) # sorting on top value 
    line_list =[]  
    word_list_len = len(word_list)
    temp=[]
    temp.append(word_list[0])   # intializing the value with pre-assumption 
    for i in range(1,word_list_len, 1):
        wa_top =word_list[i-1][2][1]
        wa_bottom = word_list[i-1][2][3]

        wb_top =word_list[i][2][1]
        wb_bottom = word_list[i][2][3]
        if wa_top <= wb_bottom and wa_bottom >= wb_top:
            temp.append(word_list[i])
        else:
            temp.sort(key=lambda sort_on_left_ : sort_on_left_[2][0]) # sorting on the left value
            line_list.append(temp)
            temp=[]
            temp.append(word_list[i])
    line_list.append(temp)
    return tuple(line_list)
       
def get_image_data(folder_path,file_name=False):
    file_path = glob.glob(folder_path)
    img_list =[]
    file_name=[]
    for path in file_path:
        if path.split('.')[-1]=='pdf':
            print('pdf file skiped')
            continue
#        print(path)
        img = cv2.imread(path)
        img_list.append(img)
        file_name.append(path)
    if file_name:
        return img_list,file_name
    return img_list

def display_image(img):
    res = isinstance(img, str)
    if res:
        img = cv2.imread(img)
        
    cv2.namedWindow('Image', cv2.WINDOW_NORMAL)
    cv2.imshow('Image', img)

def display_bounding_box(img, word_bbox_list,title="word bounding box"):
    for bbox in word_bbox_list:
        b = bbox[-1]
        img = cv2.rectangle(img, (int(b[0]), int(b[1])), (int(b[2]), int(b[3])), (0, 255, 0), 2)

    # show annotated image and wait for keypress
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    cv2.imshow(title, img)

def display_connected_graph(img,lookup,conn_type=None,isbox=False):
    
    for key in lookup.keys():    
        if isbox or conn_type==None: 
             b =  word_dict[key][1][2]
             img = cv2.rectangle(img, (int(b[0]), int(b[1])), (int(b[2]), int(b[3])), (0, 255, 0), 2)
             if conn_type==None :
                 continue
        if conn_type=='right' or conn_type=='merge': 
           
            if lookup.get(key).get('right') == None:
                continue    
            right_value = lookup[key]['right'][0]
            right_source = word_dict[key][1][2]
            right_target= word_dict[right_value][1][2] 
            y1 =sorted([right_source[1],right_source[3],right_target[1]])[1]
            y2 =sorted([right_source[1],right_source[3],right_target[3]])[1]
            y=round((y1+y2)/2)
            start_point = (right_source[2],y)
            end_point = (right_target[0],y)
#            print(start_point,end_point)
            img = cv2.line(img, (start_point),(end_point), (0, 0, 255), 2)
            
        if conn_type=='bottom'or conn_type=='merge':
            if lookup.get(key).get('bottom') == None:
                continue
            bottom_value = lookup[key]['bottom'][0]
            bottom_source = word_dict[key][1][2]
            bottom_target= word_dict[bottom_value][1][2]
#            print('bottom_source {} target {}  point stored {}'.format(bottom_source, bottom_target,word_dict[key][1]))
            x1 =sorted([bottom_source[0],bottom_target[0],bottom_target[2]])[1]
            x2 =sorted([bottom_source[2],bottom_target[0],bottom_target[2]])[1]
            x=round((x1+x2)/2)
            start_point = (x,bottom_source[3])
            end_point = (x,bottom_target[1])
#            print(start_point,end_point)
            img = cv2.line(img, (start_point),(end_point), (0, 255, 255), 2)
        
            
    cv2.namedWindow('connected node Image', cv2.WINDOW_NORMAL)
    cv2.imshow('connected node Image', img)
    cv2.waitKey(0)
            
def display_graph(adjacency_matrix, mylabels):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    nx.draw(gr, node_size=500, labels=mylabels, with_labels=True)
    plt.show()

def find_right_NN(row,col,lines,page_width,lookup):
    min_dist =99999
    RD_R=99999
    col_size = len(lines[row])
    source =lines[row][col]
#    print('source : ', source)
    source_TR= source[2][1]   # assume the co-ordinates as (y,x) and bounding box has (y,x,h,w) ->(left, top, right ,bottom)
    source_BR = source[2][3]
    for j in range(col+1, col_size):
        target = lines[row][j]
        target_TL = target[2][1]
        target_BL= target[2][3]
        
        case1 =source_TR <= target_TL <= source_BR       # case 1: when target top overlap with source  (horizontally)
        case2 =source_TR <= target_BL<= source_BR # case 2: when target bottom overlap with source in right
        case3 =target_TL < source_TR < target_BL     # case 3: when source is under overlap with target
        if case1 or case2 or case3:
            # calculaing relative distance WRT soruce and targe 
            #RD_R = left(targe)-right(soruce)/page_width
            RD_R = (target[2][0] - source[2][2])/page_width
            if(min_dist > RD_R):   # tracking minimum distance 
                if not bool(lookup.get(target[0]).get('left'))   :  # if target is empty for source in lookup 
                    lookup[source[0]]['right']=[target[0],RD_R]
                    lookup[target[0]]['left']= [source[0],RD_R]
                    min_dist=RD_R
#                    print('target : ', target)
                else:
                    if lookup[target[0]]['left'][1] > RD_R:   # if target is Not empty for source then check left distance
                        remove_source = lookup[target[0]]['left'][0]
                        lookup[target[0]]['left']= [source[0],min_dist]
                        lookup[remove_source]['right']=None
                        min_dist=RD_R
#                        print('update target : ',target)
#                        print('update remove_source : ',remove_source)
                        
                
        if RD_R > min_dist: # tracking prev_dist for stoping the seraching of NN  
            break       
    
    return lookup

def find_bottom_NN(row,col,lines,page_hight,lookup):
    min_dist =99999
    RD_B=99999
    row_size = len(lines)
    source =lines[row][col]
#    print('source : ',source)
    source_BL = source[2][0]  # assume the co-ordinates as (y,x) and bounding box has (y,x,h,w) ->(left, top, right ,bottom)
    source_BR = source[2][2]
    for i in range(row+1,row_size):
        col_size = len(lines[i]) # finding the column of the line
        for j in range(0, col_size):       
            target = lines[i][j]           
            target_TL = target[2][0]
            target_TR = target[2][2]
            case1 =source_BL <= target_TR <= source_BR  # case 1. target right overlap with source in bottom (vertically)
            case2 =source_BL <= target_TL<=source_BR   # case 2. target left overlap with source  in bottom
            case3 =target_TL < source_BL < target_TR   #case 3 . source overlap under target  in bottom
            
            if case1 or case2 or case3:    
                # calculaing relative distance WRT soruce and targe 
                #RD_B = top(targe)-bottom(soruce)/page_hight
                RD_B = (target[2][1] - source[2][3])/page_hight               
                if(min_dist > RD_B):   # tracking minimum distance 
                    if not bool(lookup.get(target[0]).get('top')) :    # if source is empty than updated in lookup 
                        min_dist=RD_B
                        lookup[source[0]]['bottom']=[target[0],min_dist]
                        lookup[target[0]]['top']=[source[0],min_dist]
#                        print('target : ',target)
                        
                    else:
                        if(lookup[target[0]]['top'][1] > RD_B):   # source is updated and checking which one having less min_dist
                            min_dist=RD_B
                            remove_source = lookup[target[0]]['top'][0]
                            lookup[target[0]]['top']=[source[0],min_dist]
                            lookup[remove_source]['bottom']=None
#                            print('update target : ',target)
#                            print('update remove_source : ',remove_source)
                        
            if RD_B > min_dist : # tracking prev_dist for stoping the seraching of NN
                i= row_size + 1 # terminating the seraching of NN in bottom 
#                 print('breaking')
                break
                
    return lookup

def get_NN_lookup_table(lines,page_width,page_hight):
    
    lookup={}
    for row,line in  enumerate(lines):
        for col,word in  enumerate(line):
            lookup[word[0]]={}

    for line_idx, line in enumerate(lines):    
        for word_idx, word in enumerate(line): 
            lookup= find_right_NN(line_idx,word_idx,lines,page_width,lookup) 
            lookup= find_bottom_NN(line_idx,word_idx,lines,page_hight,lookup)
    return lookup

def get_word_dict(lines):    
    word_dict ={}
    contain_dict={}
    word_count=0
    for line in lines:
        for word in line:
            word_dict[word[0]]=[word_count,word]
            contain_dict[word[0]]=word[1]
            word_count+=1
    return word_dict,contain_dict

def create_graph_numeric_feature(word_dict,lookup):
    word_id_list = word_dict.keys()
    size = len(word_id_list)
    adj_mat = np.zeros((size,size))
    num_feature = np.zeros((size,4))
    
    for row, word_id in enumerate(word_id_list):
        word_NN = lookup[word_id]
        
        if bool(word_NN.get('left')):
            idx , weight = word_NN.get('left')
            col = word_dict[idx][0]
            adj_mat[row][col]= -weight
            num_feature[row][0]=-weight
        
        if bool(word_NN.get('top')):
            idx , weight = word_NN.get('top')
            col = word_dict[idx][0]
            adj_mat[row][col]= -weight
            num_feature[row][1]=-weight
        
        if bool(word_NN.get('right')):
            idx , weight = word_NN.get('right')
            col = word_dict[idx][0]
            adj_mat[row][col]= weight
            num_feature[row][2]=weight
            
        if bool(word_NN.get('bottom')):
            idx , weight = word_NN.get('bottom')
            col = word_dict[idx][0]
            adj_mat[row][col]= weight
            num_feature[row][3]=weight
            
    return adj_mat,num_feature

def create_label_feature(entity):
    entity_labels=['label-0', 'label-1', 'label-2', 'label-3', 'label-4', 'label-5', 'label-6', 'label-7', 
                   'label-8', 'label-9', 'label-10', 'label-11', 
                   'label-12', 'label-13', 'label-14', 'label-15', 'label-16', 'label-17', 'label-18', 'label-19']
    
    entity=entity.lower()
    label=np.zeros((len(entity_labels)))
    try:
        label[entity_labels.index(entity)]=1
    except ValueError:
        label[19]=1
    finally:
        return label
#--------------------------------model creation ---------------------------------------------------------   
class CreateDataset(Dataset):
    def __init__(self, adj_mat_list, feature_vector_list, label_list=None):
        self.adj_mat =adj_mat_list
        self.feature=feature_vector_list
        self.label= label_list
    def __len__(self):
        return len(self.feature)

    def __getitem__(self, idx):
        A = csr_matrix(self.adj_mat[idx])
        edge_index, edge_attr = convert.from_scipy_sparse_matrix(A)
        x=torch.tensor(self.feature[idx], dtype=torch.float)
        
        if self.label != None:
            y=torch.tensor(self.label[idx], dtype=torch.long)
            data =Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
        else:
            data =Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
            
        return data
        
class Net(torch.nn.Module):
    def __init__(self,features_size):
        super(Net, self).__init__()
        self.conv1 = ChebConv(features_size, 16,3)
        self.conv2 = ChebConv(16, 32,3)
        self.conv3 = ChebConv(32, 64,3)
        self.conv4 = ChebConv(64, 128,3)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        return F.log_softmax(x, dim=1)    
    
def get_validation_loss(validloder, model, loss_fun, epoch,betch_size=2, train_on_gpu=False):
     #validation loss     
    valid_loss = 0.0
    valid_acc = 0.0

    # Don't need to keep track of gradients
    with torch.no_grad():
        # Set to evaluation mode
        model.eval()
        # Validation loop
        for data in validloder:
            # Tensors to gpu
            if train_on_gpu:
                data= data.cuda()
            # Forward pass
            output = model(data)
#             print('output size {}:\n{} '.format(output.size(),output))
            # Validation loss
            loss = loss_fun(output, data.y.long())
#             pred = output.max(dim=1)[1]
            # Multiply average loss times the number of examples in batch
            valid_loss += loss.item() * betch_size

            # Calculate validation accuracy
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(data.y.view_as(pred))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples
            valid_acc += accuracy.item() * betch_size

        # Calculate average losses
        valid_loss = valid_loss / len(validloder.dataset)

        # Calculate average accuracy
        valid_acc = valid_acc / len(validloder.dataset)
        
#         print(f'\nEpoch: {epoch} \tValidation Loss: {valid_loss:.4f} \tValidation Accuracy: {100 * valid_acc:.2f}')
        
    return valid_loss, valid_acc

def get_taining_loss(trainloader,model, optimizer,loss_fun,epoch,betch_size=2,train_on_gpu=False):
    # keep track of training and validation loss each epoch
    train_loss = 0.0
    train_acc = 0
    
    # Set to training mode
    model.train()
    
    # trainig loop 
    for i, data in enumerate(trainloader, 0):

        # Tensors to gpu
        if train_on_gpu:
            print('running on gpu')
            data = data.cuda()
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = model(data)
#         print(type(output))
        loss = loss_fun(output, data.y.long())
        loss.backward()
        optimizer.step()

        # Track train loss by multiplying average loss by number of examples in batch
        train_loss += loss.item() * betch_size

        # Calculate accuracy by finding max log probability
        _, pred = torch.max(output, dim=1)
        correct_tensor = pred.eq(data.y.view_as(pred))
        # Need to convert correct tensor from int to float to average
        accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
       
        # Multiply average accuracy times the number of examples in batch
        train_acc += accuracy.item() * betch_size
        
        # Calculate average losses
        train_loss = train_loss / len(trainloader)
       
        # Print training and validation results
                
#         print(f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tTraining Accuracy: {100 * train_acc:.2f}')
    return train_loss, train_acc

def train_model(model,model_path,trainloder,validloder,No_epochs=2000,max_epochs_stop=50,print_every=100):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    Entropy_loss = nn.CrossEntropyLoss()
    
    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf
    history = []
    overall_start=timer()
    for epoch in range(No_epochs):
        start=timer()
        train_loss, train_acc = get_taining_loss(trainloder,model,optimizer,Entropy_loss,epoch)
        
        valid_loss, valid_acc = get_validation_loss(validloder,model,Entropy_loss,epoch)
        
        history.append([train_loss, valid_loss, train_acc, valid_acc])  # adding data for plot
        
        # Print training and validation results
        if (epoch + 1) % print_every == 0:
            print(f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \t\tValidation Loss: {valid_loss:.4f}')
#             print(f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\tValidation Accuracy: {100 * valid_acc:.2f}%')
            print(f'\t\t\t( {timer() - start:.2f} seconds elapsed in epoch {epoch})',end='\r')
        
        # Save the model if validation loss decreases
        if valid_loss < valid_loss_min:
            # Save model
            torch.save(model.state_dict(), model_path)
            # Track improvement
            epochs_no_improve = 0
            valid_loss_min = valid_loss
#            valid_best_acc = valid_acc
            best_epoch = epoch
        # Otherwise increment count of epochs with no improvement
        else:
            epochs_no_improve += 1    
            
        # Trigger early stopping
        if epochs_no_improve >= max_epochs_stop or epoch+1 >= No_epochs:
            if( epoch+1 >= No_epochs):
                print(f'\n-----------------------------------Traning completed----------------------------------')
                print(f'\nCompleted Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%')
            else:
                print(f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%')
                
            total_time = timer() - overall_start
            print(f'\t({total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch)')

            # Load the best state dict
            model.load_state_dict(torch.load(model_path))
            # Attach the optimizer
            model.optimizer = optimizer

            # Format history
            history = pd.DataFrame(history,columns=['train_loss', 'valid_loss', 'train_acc','valid_acc'])
            return model, history

def plot_loss(history):
    plt.figure(figsize=(8, 6))
    for c in ['train_loss', 'valid_loss']:
        plt.plot(history[c], label=c)
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Average Negative Log Likelihood')
    plt.title('Training and Validation Losses')
    
def plot_accuracy(history):
    plt.figure(figsize=(8, 6))
    for c in ['train_acc', 'valid_acc']:
        plt.plot(100 * history[c], label=c)
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Average Accuracy')
    plt.title('Training and Validation Accuracy')

def image_feature_processing(img_data=None,folder_path=None):
    if img_data==None:
        img_data,file_name = get_image_data(folder_path,True)
    
    adj_mat_list=[]
    feature_vector_list=[]
    word_info_list=[]
    for idx,img in enumerate(img_data): 
        if folder_path !=None:
            print('processing file (id {}) : {}'.format(idx,file_name[idx]))
            
        print('processing file id : {}'.format(idx))
        word_bbox_list, bbox_width =  extract_bbox_info(img)
        word_list = remove_noise(bbox_width,word_bbox_list)
        lines = Line_Formation(word_list)
        word_dict , contain_dict= get_word_dict(lines)
        word_info_list.append([word_list,word_dict,contain_dict])
        width, hight ,_ = img.shape
        lookup= get_NN_lookup_table(lines,width,hight)
        adj_mat, num_feature = create_graph_numeric_feature(word_dict,lookup)
        feature_vector = create_feature_vector(word_dict,num_feature)
        adj_mat_list.append(adj_mat)
        feature_vector_list.append(feature_vector)
        
    return adj_mat_list,feature_vector_list,word_info_list

def print_ocr_string(img):  
    res = isinstance(img, str)
    if res:
        print('reading image ..\n' )
        img = cv2.imread(img)  
    text = pytesseract.image_to_string(img)
#     print(text)
    return text


In [3]:
#--------------------------------main ---------------------------------------------
    
    
pincode_path="C:/Users/Sanjeev/Documents/Python Scripts/cheat Sheet/Pincode_30052019.csv"
# pincode_path= "C:/Users/Sanjeev/Documents/Accenture office/Pincode_30052019.csv"
# country_path="C:/Users/Sanjeev/Documents/Accenture office/countries.csv"
country_path="C:/Users/Sanjeev/Documents/Accenture office/countries.txt"
database_path ="C:/Users/Sanjeev/Documents/Accenture office/database"
img_path="C:/Users/Sanjeev/Documents/Accenture office/invoice3.png"
tesseract_path ='C:/python_lib/ocr/tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = 'C:/python_lib/ocr/tesseract.exe'
model_path = "C:/Users/Sanjeev/Documents/Accenture office/invoice_model"
folder_path = "C:/Users/Sanjeev/Documents/Accenture office/invoice data/*"

# folder_path = "C:/Users/Sanjeev/Documents/Python Scripts/Dataset/output/*.png"
csv_path ='C:/Users/Sanjeev/Documents/Python Scripts/Dataset/csv/'
data_path = "C:/Users/Sanjeev/Documents/Python Scripts/Dataset/df_word_list"

In [4]:
# reading the image file from the folder and stored as numpy array
img_data, file_name = get_image_data(folder_path,True)
print(len(img_data))
#1,2,3,5  (issue in noise removal-> 4)
# del img_data[4]
# del file_name[4]
print(len(img_data))

pdf file skiped
pdf file skiped
6
6


In [5]:
# pre-processing the image data 
# 1. feed the image to OCR and get HOCR file which contain bounding box infomation
# 2. Create numeric, boolean, and word features 
# 3. Create Graph 
# 4. return adjacency metric A, feature vector 317 dim, and word infoamtion 
adj_mat_list,feature_vector_list, word_info_list = image_feature_processing(img_data=img_data[0:3])    # pre-processing and graph Creation

processing file id : 0
processing file id : 1
processing file id : 2


In [6]:
# creating label for the each node of the graph (dummy label for testing Neural Network pipeline)
label_list=[]
seed=1234567833
np.random.seed(seed)
for i in range(len(adj_mat_list)):
    label = np.random.randint(0,10,adj_mat_list[i].shape[0])
    label_list.append(label)


In [7]:
# creating dataset and dataloader for pytorch pipeline
dataset =CreateDataset(adj_mat_list,feature_vector_list,label_list)

betch_size=2
train_dataloder = DataLoader(dataset,betch_size)
torch.manual_seed(seed)
model=Net(317)
model , history = train_model(model,model_path,train_dataloder,train_dataloder,10,5,10) # train the Neural Network 



Epoch: 9 	Training Loss: 40.6291 		Validation Loss: 43.2771
			( 0.10 seconds elapsed in epoch 9)
-----------------------------------Traning completed----------------------------------

Completed Total epochs: 9. Best epoch: 9 with loss: 43.28 and acc: 21.75%
	(1.37 total seconds elapsed. 0.14 seconds per epoch)


In [ ]:
# ploting training and validation loss 
plot_loss(history)

# ploting training and validation accuracy 
# plot_accuracy(history)

In [ ]:
# testing he model after training 

## yet to impement

In [ ]:
# visualization of image, word detected by ocr, words after removal of noise, and Graph generated.

idx=1
img=cv2.imread(file_name[idx])
display_image(img)

word_bbox_list, bbox_width =  extract_bbox_info(img)
display_bounding_box(img, word_bbox_list,'orginal words')

# print_ocr_string(img)

word_list = remove_noise(bbox_width,word_bbox_list)
img=cv2.imread(file_name[idx])
display_bounding_box(img, word_list,'word after removal')

lines = Line_Formation(word_list)
word_dict , contain_dict= get_word_dict(lines)
width, hight ,_ = img.shape
lookup= get_NN_lookup_table(lines,width,hight)
adj_mat, num_feature = create_graph_numeric_feature(word_dict,lookup)

img = cv2.imread(file_name[idx])
display_connected_graph(img, lookup, 'merge',True)

cv2.waitKey(0)
# feature_vector = create_feature_vector(word_dict,num_feature)

In [ ]:
img = cv2.imread(file_name[idx])
display_connected_graph(img, lookup, 'merge',True)

In [ ]:
lines


In [ ]:
No_feature=317
def get_prediction(model,img_list=None,img_path=None):
#     model.eval()
    if img_list != None:
        img_data= img_list
    if img_path !=None:
        img_data=img_path
    adj_mat_list,feature_vector_list, word_info_list = image_feature_processing(img_data=img_data)
    
    betch_size = len(adj_mat_list)   # no of data for testing 
    
    if betch_size==1:
        print('betch_size : ',betch_size)
        test_x=torch.tensor(feature_vector_list[0], dtype=torch.float) 
        test_edge_index, test_edge_attr = convert.from_scipy_sparse_matrix(csr_matrix(adj_mat_list[0]))
        test_data=Data(x=test_x,edge_index=test_edge_index,edge_attr=test_edge_attr)
    else:
        print('betch_size : ',betch_size)
        test_data =CreateDataset(adj_mat_list, feature_vector_list)
        test_data = DataLoader(test_data,betch_size)
        test_data = next(iter(test_data))

    pred_out = model(test_data) 
    print(pred_out.size())
    _ ,pred_out = torch.max(pred_out, dim=1)
    
    return pred_out
            
    
    

In [ ]:
test_model.load_state_dict(torch.load(model_path))
test_model.eval()
# pred =get_prediction(test_model,img_data)
# print(pred.size())
# correct_tensor = pred.eq(test_y)
# print(sum(correct_tensor))
# print(correct_tensor)

In [ ]:
p_o=pred.numpy()
p_o[0:5]

In [ ]:
sum=0
for label in label_list:
    print(len(label))
    sum+= len(label)
sum

In [ ]:
test_idx=1

test_x=torch.tensor(feature_vector_list[test_idx], dtype=torch.float)
test_y=torch.tensor(label_list[test_idx], dtype=torch.long)
# A = csr_matrix(adj_mat_list[test_idx])
test_edge_index, test_edge_attr = convert.from_scipy_sparse_matrix(csr_matrix(adj_mat_list[test_idx]))
test_data=Data(x=test_x,edge_index=test_edge_index,edge_attr=test_edge_attr)
test_model = Net(317)
test_model.load_state_dict(torch.load(model_path))
test_model.eval()


In [ ]:
pred_out= test_model(test_data)

In [ ]:
a, pred = torch.max(pred_out, dim=1)
correct_tensor = pred.eq(test_y)
# correct_tensor.numpy()

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

text =f'sanjeev has good heart person and he is nice.His DOB is 15/05/1992.'
# text =text.split(' ')[3]
doc=nlp(text)
# print(doc)

# for token in doc:
#     print('{} -> {}'.format(token.text, token.pos_))

for token in doc.ents:
    print('{} -> {}'.format(token.text, token.label_))

In [ ]:
#this function takes input word and out p
# ORG -> [1,0,0,0]
# Date -> [0,1,0,0]
# GPE ->  [0,0,1,0]
# MONEY/CARDINAL->[0,0,0,1]
#except this -> [0,0,0,0]

def get_ner_tag(ner_tag):
    if len(ner_tag) is not 0:
        ner_tag=ner_tag[0].label_
        print(ner_tag)
        if ner_tag=='ORG':
            return [1,0,0,0]
        elif ner_tag=='DATE':
            return [0,1,0,0]
        elif ner_tag=='GPE':
            return [0,0,1,0]
        elif ner_tag=='MONEY' or ner_tag=='CARDINAL':
            return [0,0,0,1]
        else:
            return [0,0,0,0]
    else:
         return [0,0,0,0]
    
    

#this function takes token/word as input and give 2-d output( 10: -> noun/pronoun,01-> Number, 00-> others )
# [1,0,0]

def get_pos_tag(pos_tags):
    
    if pos_tags =='NOUN' or pos_tags=='PRON':
        return [1,0]
    elif pos_tags =='NUM':
        return [0,1]
    else:
        return [0,0]
        
    

def create_tag_features(word_list):
    nlp = spacy.load("en_core_web_sm")
    pos_tag_list = []
    ner_tag_list=[]
    for word in word_list:
        tag =nlp(word)
        pos_tag_list.append(get_pos_tag(tag[0].pos_))
        ner_tag_list.append(get_ner_tag(tag.ents))
    return pos_tag_list , ner_tag_list
    
    

In [ ]:
print(text)
pos, ner =create_tag_features(text.split(' '))

In [ ]:
print(pos,ner)


In [ ]:
idx=0
img=cv2.imread(file_name[idx])
# display_image(img)

word_bbox_list, bbox_width =  extract_bbox_info(img)
# word_bbox_list =noise_remove(word_bbox_list)
display_bounding_box(img, word_bbox_list,'orginal words')
cv2.waitKey(0)

In [ ]:
def noise_remove(word_list,skip_symbol=None):
    filter_list=[]
    for w_list in word_list:
        word= w_list[1]
        if len(word)==1: # checking if word length is 1 it could be noise
            if skip_symbol and word in skip_symbol:   # if word is in our desired symbol ex:- $,₹, etc 
                filter_list.append(w_list)
            elif is_alphaNumeric(word):   # consider the case John D :- d has len one but it is alphabet.
                filter_list.append(w_list)
        else:
            filter_list.append(w_list)
    return filter_list    # this sshould be the main noise removal
            

In [40]:
def row_clustring(lines_list, row_threshold=4):
    row_bbox_list =[]
    test_list=[]
    merge_info=[] #  for storing the merge info
    for idx,line in enumerate(lines_list):
        temp_test_list=[]
        curr_bbox_id=0
        next_bbox_id=1
        prev_id=-1
        total_word=len(line) 
        print(line)
        print('total word = ',total_word)
        temp_merge=[]
        info=[]
#         temp_merge.append(line[curr_bbox_id])
        curr_bbox=line[curr_bbox_id][2]
        print('curr_bbox : ',curr_bbox)
        
        while next_bbox_id < total_word:
            next_bbox=line[next_bbox_id][2]
            print('next_bbox : ',line[next_bbox_id])
            dist = np.sqrt((curr_bbox[2] - next_bbox[0])**2)    # calculating distance having y point common(mid point in overlaping case)
#             dist = curr_bbox[2] - next_bbox[0] 
            print('dist ',dist)
            if dist <= row_threshold:    # merging two bbox
#                 print('min ----------------------dist-: ',dist)
                curr_bbox[2]= max(curr_bbox[2],next_bbox[2])
                curr_bbox[3]= max(curr_bbox[3],next_bbox[3])
                print('curr_bbox (merging)',curr_bbox)
                if prev_id != curr_bbox_id:
                    info.append(line[curr_bbox_id])
                    prev_id=curr_bbox_id
                info.append(line[next_bbox_id])      
            else:
                row_bbox_list.append(curr_bbox)
                print('row_bbox_list (inserting)',curr_bbox)
                temp_test_list.append(curr_bbox)  # tesing 
                
                temp_merge.append(info)
                curr_bbox_id= next_bbox_id
                curr_bbox=next_bbox
                print('curr_bbox (adding)',curr_bbox)
                info=[]
            next_bbox_id +=1
            print('next_bbox_id ', next_bbox_id)
       
        merge_info.append(temp_merge)
        if dist > row_threshold:
            row_bbox_list.append(line[next_bbox_id-1][2])
            print('row_bbox_list (inserting) after last word',line[next_bbox_id-1])
            temp_test_list.append(line[next_bbox_id-1][2])
        else:
            row_bbox_list.append(curr_bbox)
            print('row_bbox_list (inserting) after merge',curr_bbox)
            temp_test_list.append(curr_bbox)
        test_list.append(temp_test_list)
        print('---------------------------line {}-----------------------------'.format(idx+1))
    return row_bbox_list,merge_info,test_list
                      

In [9]:
def check_overlaping(source,target):
    source_BL = source[0]  # assume the co-ordinates as (y,x) and bounding box has (y,x,h,w) ->(left, top, right ,bottom)
    source_BR = source[2]           
    target_TL = target[0]
    target_TR = target[2]
    
    
    case1 =source_BL <= target_TR <= source_BR  # case 1. target right overlap with source in bottom (vertically)
    case2 =source_BL <= target_TL<=source_BR   # case 2. target left overlap with source  in bottom
    case3 =target_TL < source_BL < target_TR   #case 3 . source overlap under target  in bottom

    if case1 or case2 or case3: 
        return True
    else:
        return False
# we always check from top row to next bottom row for merging two boxes.             
def col_clestring(row_merge_list, col_threshold):
    print('enter for calculaton ')
    merge_list =[]
    merge_info=[]
    top_row= row_merge_list[0]
    for idx in range(1,len(row_merge_list)):
        print('loop ', idx)
        bottom_row = row_merge_list[idx]
        top_row_len, bottom_row_len = len(top_row), len(bottom_row)
        i, j,  candidate_row=0,0,[]
#         print('top_row ', top_row)
#         print('bottom_row ', bottom_row)
#         print('len of rows top ={} ,botttom {} , i= {}, j= {}'.format(top_row_len,bottom_row_len, i, j))
        merge_flag=False
        while i < top_row_len:
            while j < bottom_row_len:
                if (i>= top_row_len):
                    print('breaking top row loop')
                    break
                top_bbox = top_row[i]
                bottom_bbox= bottom_row[j]
                
                print('top_row ', top_row)
                print('bottom_row ', bottom_row)
                print('len of rows top ={} ,botttom {} , i= {}, j= {}'.format(top_row_len,bottom_row_len, i, j))
                print('top_bbox = {} \nbottom_bbox = {}'.format(top_bbox,bottom_bbox) )
                if check_overlaping(top_bbox,bottom_bbox):    # when top_bbox and bottom_bbox overlap each other 
                    print('overlaping  ')
                    dist = np.sqrt(( bottom_bbox[1] - top_bbox[3])**2)
                    print('dist : ', dist)
                    if dist <=col_threshold:
                        print('distance statisfy : ',j)
                        top_bbox[0]= min(top_bbox[0],bottom_bbox[0])
                        top_bbox[1]= min(top_bbox[1],bottom_bbox[1])
                        top_bbox[2]= max(top_bbox[2],bottom_bbox[2])
                        top_bbox[3]= max(top_bbox[3],bottom_bbox[3])
                        j +=1
                        merge_flag=True
                        
                        if j >= bottom_row_len:  # handling last box box when it merge 
                            candidate_row.append(top_bbox)
                            print('adding *last* candidate row (top box)  ', top_bbox)
                        
                    else:
                        print('distance not statisfy: ', j)
                        candidate_row.append(bottom_bbox)
                        print('adding to candidate row (bottom_bbox) ', bottom_bbox)
                        j +=1
                else:
                    print('Not overlaping  ')
                    if(top_bbox[2] < bottom_bbox[0]):    # bottom_bbox away(right most) from top_bbox 
                        if merge_flag:
                            candidate_row.append(top_bbox)
                            print('(right most) adding to candidate row (top box) ', top_bbox)
                        merge_list.append(top_bbox)
                        print('(right most)adding to mergeList  box ', top_bbox)
                        i +=1
                    else:
                        candidate_row.append(bottom_bbox)
                        print('(left most) adding to candidate row (bottom_bbox) ', bottom_bbox)
                        j += 1
                    
                print('value i= {} and j ={} '.format(i,j))
                print('merge_list : ', merge_list)
                print('candidate_row :',candidate_row)
                print('\n--exsiting the while 1 loop--\n') 
            
            while(j < bottom_row_len): # case when top row empty and bottom row has word 
                print('bottom bbox is not empty : ', j)
                candidate_row.append(bottom_row[j])
                j +=1
                print('value i= {} and j ={} '.format(i,j))
            while(i < top_row_len): # case when bottom row empty and top row has word 
                print('top box  is not empty : ', i)
                merge_list.append(top_row[i])
                i +=1
                print('value i= {} and j ={} '.format(i,j))
        print('exsiting the while 2 loop')
        print('merge_list : ', merge_list)
        print('candidate_row :',candidate_row)
        top_row = candidate_row
        print('candidate_row ', candidate_row)
        print('\n---------------------------------------------------------------\n')
#     merge_list.append(top_row)
    return  merge_list+top_row
            

In [ ]:
d = col_clestring(c, 10)


In [ ]:
d[-1]

In [26]:
# visualization of image, word detected by ocr, words after removal of noise, and Graph generated.

idx=1
img=cv2.imread(file_name[idx])
# display_image(img)

word_bbox_list, bbox_width =  extract_bbox_info(img)
# display_bounding_box(img, word_bbox_list,'orginal words')

# print_ocr_string(img)

word_list = remove_noise(bbox_width,word_bbox_list)
img=cv2.imread(file_name[idx])
# display_bounding_box(img, word_list,'word after removal')

lines = Line_Formation(word_list)
word_dict , contain_dict= get_word_dict(lines)
width, hight ,_ = img.shape
lookup= get_NN_lookup_table(lines,width,hight)
adj_mat, num_feature = create_graph_numeric_feature(word_dict,lookup)

img = cv2.imread(file_name[idx])
# display_connected_graph(img, lookup, 'merge',True)

cv2.waitKey(0)
# feature_vector = create_feature_vector(word_dict,num_feature)

-1

In [12]:
lines

([['word_1_4', '"Romashka"', [49, 44, 120, 54]],
  ['word_1_5', 'Ltd.', [125, 44, 145, 54]],
  ['word_1_1', 'Invoice', [411, 34, 460, 45]],
  ['word_1_2', 'ID:', [465, 34, 483, 45]],
  ['word_1_3', 'INV/20111209-22', [567, 34, 679, 45]]],
 [['word_1_6', '1600', [49, 60, 77, 70]],
  ['word_1_7', 'Amphitheatre', [81, 60, 159, 73]],
  ['word_1_8', 'Parkway', [164, 60, 213, 73]],
  ['word_1_9', 'Mountain', [217, 60, 270, 70]],
  ['word_1_10', 'View,', [275, 60, 306, 72]],
  ['word_1_11', 'CA', [311, 60, 328, 70]],
  ['word_1_12', '94043', [332, 60, 368, 70]],
  ['word_1_13', 'Invoice', [411, 59, 460, 70]],
  ['word_1_14', 'date:', [465, 59, 498, 70]],
  ['word_1_15', '12/08/2011', [567, 58, 635, 69]]],
 [['word_1_16', 'Due', [411, 83, 438, 94]],
  ['word_1_17', 'date:', [442, 83, 476, 94]],
  ['word_1_18', '12/25/2012', [567, 83, 638, 94]]],
 [['word_1_19', 'Text', [411, 108, 440, 119]],
  ['word_1_20', 'custom', [444, 109, 495, 119]],
  ['word_1_21', 'fiel', [500, 108, 520, 119]],
  ['wor

In [17]:
lines

([['word_1_4', '"Romashka"', [49, 44, 145, 54]],
  ['word_1_5', 'Ltd.', [125, 44, 145, 54]],
  ['word_1_1', 'Invoice', [411, 34, 483, 45]],
  ['word_1_2', 'ID:', [465, 34, 483, 45]],
  ['word_1_3', 'INV/20111209-22', [567, 34, 679, 45]]],
 [['word_1_6', '1600', [49, 60, 368, 73]],
  ['word_1_7', 'Amphitheatre', [81, 60, 159, 73]],
  ['word_1_8', 'Parkway', [164, 60, 213, 73]],
  ['word_1_9', 'Mountain', [217, 60, 270, 70]],
  ['word_1_10', 'View,', [275, 60, 306, 72]],
  ['word_1_11', 'CA', [311, 60, 328, 70]],
  ['word_1_12', '94043', [332, 60, 368, 70]],
  ['word_1_13', 'Invoice', [411, 59, 498, 70]],
  ['word_1_14', 'date:', [465, 59, 498, 70]],
  ['word_1_15', '12/08/2011', [567, 58, 635, 69]]],
 [['word_1_16', 'Due', [411, 83, 476, 94]],
  ['word_1_17', 'date:', [442, 83, 476, 94]],
  ['word_1_18', '12/25/2012', [567, 83, 638, 94]]],
 [['word_1_19', 'Text', [411, 108, 520, 119]],
  ['word_1_20', 'custom', [444, 109, 495, 119]],
  ['word_1_21', 'fiel', [500, 108, 520, 119]],
  ['wo

In [27]:
# a,b = row_merge(lines,30)
a,b,c = row_clustring(lines,10)
# idx=1
display_bounding_box(img, word_bbox_list,'orginal words')
img=cv2.imread(file_name[idx])
display_bounding_box(img, a, 'after row clusting' )
# d= col_clestring(c,10)
# img=cv2.imread(file_name[idx])
# display_bounding_box(img, d,'after col clusting' )
cv2.waitKey(0)

[['word_1_4', '"Romashka"', [49, 44, 120, 54]], ['word_1_5', 'Ltd.', [125, 44, 145, 54]], ['word_1_1', 'Invoice', [411, 34, 460, 45]], ['word_1_2', 'ID:', [465, 34, 483, 45]], ['word_1_3', 'INV/20111209-22', [567, 34, 679, 45]]]
total word =  5
curr_bbox :  [49, 44, 120, 54]
next_bbox :  ['word_1_5', 'Ltd.', [125, 44, 145, 54]]
dist  5.0
curr_bbox (merging) [49, 44, 145, 54]
next_bbox_id  2
next_bbox :  ['word_1_1', 'Invoice', [411, 34, 460, 45]]
dist  266.0
row_bbox_list (inserting) [49, 44, 145, 54]
curr_bbox (adding) [411, 34, 460, 45]
next_bbox_id  3
next_bbox :  ['word_1_2', 'ID:', [465, 34, 483, 45]]
dist  5.0
curr_bbox (merging) [411, 34, 483, 45]
next_bbox_id  4
next_bbox :  ['word_1_3', 'INV/20111209-22', [567, 34, 679, 45]]
dist  84.0
row_bbox_list (inserting) [411, 34, 483, 45]
curr_bbox (adding) [567, 34, 679, 45]
next_bbox_id  5
row_bbox_list (inserting) after last word ['word_1_3', 'INV/20111209-22', [567, 34, 679, 45]]
---------------------------line 1-------------------

curr_bbox (merging) [83, 288, 184, 298]
next_bbox_id  3
next_bbox :  ['word_1_47', 'invoices', [188, 288, 235, 298]]
dist  4.0
curr_bbox (merging) [83, 288, 235, 298]
next_bbox_id  4
next_bbox :  ['word_1_48', 'list', [239, 288, 255, 298]]
dist  4.0
curr_bbox (merging) [83, 288, 255, 298]
next_bbox_id  5
row_bbox_list (inserting) after merge [83, 288, 255, 298]
---------------------------line 7-----------------------------
[['word_1_49', '2', [48, 318, 55, 328]], ['word_1_50', 'Develop', [76, 318, 123, 331]], ['word_1_51', 'x17.0', [412, 318, 443, 328]], ['word_1_52', 'hours', [464, 318, 496, 328]], ['word_1_53', '40.00', [587, 318, 619, 328]], ['word_1_54', '680.00', [669, 318, 708, 328]]]
total word =  6
curr_bbox :  [48, 318, 55, 328]
next_bbox :  ['word_1_50', 'Develop', [76, 318, 123, 331]]
dist  21.0
row_bbox_list (inserting) [48, 318, 55, 328]
curr_bbox (adding) [76, 318, 123, 331]
next_bbox_id  2
next_bbox :  ['word_1_51', 'x17.0', [412, 318, 443, 328]]
dist  289.0
row_bbox_lis

[['word_1_97', 'Total', [410, 562, 445, 573]], ['word_1_98', '(EUR):', [450, 562, 494, 576]], ['word_1_99', '901.80', [665, 562, 709, 573]]]
total word =  3
curr_bbox :  [410, 562, 445, 573]
next_bbox :  ['word_1_98', '(EUR):', [450, 562, 494, 576]]
dist  5.0
curr_bbox (merging) [410, 562, 494, 576]
next_bbox_id  2
next_bbox :  ['word_1_99', '901.80', [665, 562, 709, 573]]
dist  171.0
row_bbox_list (inserting) [410, 562, 494, 576]
curr_bbox (adding) [665, 562, 709, 573]
next_bbox_id  3
row_bbox_list (inserting) after last word ['word_1_99', '901.80', [665, 562, 709, 573]]
---------------------------line 19-----------------------------
[['word_1_100', 'Your', [42, 612, 69, 622]], ['word_1_101', 'billing', [74, 612, 106, 625]], ['word_1_102', 'information', [110, 612, 174, 622]], ['word_1_103', '(Bank,', [179, 612, 216, 625]], ['word_1_104', 'Address,', [220, 612, 272, 624]], ['word_1_105', 'IBAN,', [277, 612, 310, 624]], ['word_1_106', 'SWIFT', [314, 612, 355, 622]], ['word_1_107', '&',

-1

In [18]:
b

[[[['word_1_4', '"Romashka"', [49, 44, 145, 54]],
   ['word_1_5', 'Ltd.', [125, 44, 145, 54]]],
  [['word_1_1', 'Invoice', [411, 34, 483, 45]],
   ['word_1_2', 'ID:', [465, 34, 483, 45]]]],
 [[['word_1_6', '1600', [49, 60, 368, 73]],
   ['word_1_7', 'Amphitheatre', [81, 60, 159, 73]],
   ['word_1_8', 'Parkway', [164, 60, 213, 73]],
   ['word_1_9', 'Mountain', [217, 60, 270, 70]],
   ['word_1_10', 'View,', [275, 60, 306, 72]],
   ['word_1_11', 'CA', [311, 60, 328, 70]],
   ['word_1_12', '94043', [332, 60, 368, 70]]],
  [['word_1_13', 'Invoice', [411, 59, 498, 70]],
   ['word_1_14', 'date:', [465, 59, 498, 70]]]],
 [[['word_1_16', 'Due', [411, 83, 476, 94]],
   ['word_1_17', 'date:', [442, 83, 476, 94]]]],
 [[['word_1_19', 'Text', [411, 108, 520, 119]],
   ['word_1_20', 'custom', [444, 109, 495, 119]],
   ['word_1_21', 'fiel', [500, 108, 520, 119]]]],
 [[],
  [],
  [],
  [],
  [['word_1_32', 'Unit', [520, 234, 620, 263]],
   ['word_1_33', 'price', [550, 234, 583, 263]],
   ['word_1_34', 

In [39]:
c[0],b[7]

([[49, 44, 145, 54], [411, 34, 483, 45], [567, 34, 679, 45]],
 [[], [], [], [], []])

In [10]:
def display_bounding_box(img, word_bbox_list,title="word bounding box"):
    for bbox in word_bbox_list: 
        if len(bbox)==3:
            b = bbox[-1]
#             print(bbox)
        else:
            b=bbox
        img = cv2.rectangle(img, (int(b[0]), int(b[1])), (int(b[2]), int(b[3])), (0, 255, 0), 2)

    # show annotated image and wait for keypress
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    cv2.imshow(title, img)

In [ ]:
p="C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/0325updated.task2train(626p)/X00016469612.jpg"
def print_ocr_string(img):  
    res = isinstance(img, str)
    if res:
        print('reading image ../n' )
        img = cv2.imread(img)  
    text = pytesseract.image_to_string(img)
#     print(text)
    return text

In [ ]:
print_ocr_string(p)

In [ ]:
pip list -v

In [ ]:
pip install bpemb